<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AGENTIC_OBSERVABILITY_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RUN1

In [3]:
import time
import json
from openai import OpenAI
from typing import Dict, Any

# --- CONFIGURATION (Direct Key Retrieval) ---

# Assuming this runs in a Google Colab environment or similar setup
# where the key is available via `userdata.get()`.
from google.colab import userdata

try:
    # Use the user-provided method to get the actual key
    API_KEY = userdata.get("DEEPSEEK_API_KEY")

    if not API_KEY:
        raise ValueError("DEEPSEEK_API_KEY not found in Colab userdata.")

    # Initialize the DeepSeek-compatible client
    client = OpenAI(api_key=API_KEY, base_url="https://api.deepseek.com")
    print("✅ DeepSeek Client Initialized successfully.")

except Exception as e:
    print(f"❌ Error initializing client: {e}")
    print("Please ensure you have run `pip3 install openai` and the 'DEEPSEEK_API_KEY' secret is correctly set.")
    client = None

# --- 1. The Agent Class ---
class DeepSeekReasonerAgent:
    """A minimal agent that uses DeepSeek-Reasoner to solve a task and logs observability metrics."""

    def __init__(self, client: OpenAI, agent_id="Reasoner-001"):
        self.client = client
        self.id = agent_id

    def run_task(self, request: str, system_prompt: str) -> Dict[str, Any]:
        """Executes the LLM call and captures observability metrics."""

        # --- (A) Execution & Metric Collection ---
        start_time = time.time()

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": request},
        ]

        agent_output = "API call failed."
        obs_data = {}

        try:
            # 💡 The actual call to the DeepSeek API
            response = self.client.chat.completions.create(
                model="deepseek-reasoner",
                messages=messages,
                stream=False
            )

            end_time = time.time()
            # Calculate latency
            inference_latency_ms = (end_time - start_time) * 1000

            # Extract key metrics directly from the response object
            usage = response.usage

            # The agent's core output
            final_content = response.choices[0].message.content

            # The DeepSeek Reasoner model may include a 'reasoning_content' attribute
            # We check for its presence to track the agent's decision path/CoT usage.
            reasoning = getattr(response.choices[0].message, 'reasoning_content', 'N/A')

            agent_output = final_content

            # --- (B) Structured Observability Data Payload ---
            obs_data = {
                "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
                "agent_id": self.id,
                "request_summary": request[:50] + "...",
                "inference_latency_ms": round(inference_latency_ms, 2),

                # --- Crucial LLM Observability Metrics (Cost Monitoring) ---
                "tokens_input": usage.prompt_tokens,
                "tokens_output": usage.completion_tokens,
                "tokens_total": usage.total_tokens,

                # --- Reasoning/Governance Metrics ---
                "model_name": response.model,
                "had_reasoning_step": reasoning != 'N/A', # Tracks if Chain-of-Thought was used
                "response_quality_evaluation": "Requires Human or Automated Grading",
                "reasoning_trace_length": len(reasoning) if reasoning != 'N/A' else 0,
            }

        except Exception as e:
            end_time = time.time()
            inference_latency_ms = (end_time - start_time) * 1000
            agent_output = f"API Error: {str(e)}"

            # Log failure metrics
            obs_data = {
                "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
                "agent_id": self.id,
                "request_summary": request[:50] + "...",
                "inference_latency_ms": round(inference_latency_ms, 2),
                "status": "FAILURE",
                "error_message": str(e),
                "tokens_total": 0,
            }

        return {
            "agent_output": agent_output,
            "observability_data": obs_data
        }

# --- 2. Observability Logging Function ---
def log_agent_event(data: Dict[str, Any]):
    """Prints the structured observability data in a readable JSON format."""
    print("\n" + "="*70)
    print("✨ AGENT OBSERVABILITY TRACE CAPTURED (DeepSeek Agent) ✨")
    print("="*70)
    # Use JSON format for easy ingestion by monitoring tools
    print(json.dumps(data, indent=4))
    print("="*70)

# --- 3. Demo Execution ---
if __name__ == "__main__":
    if not client:
        print("\nDemo cannot run. Client initialization failed due to missing API key or connection error.")
    else:
        my_agent = DeepSeekReasonerAgent(client=client)

        # --- Scenario: Logic Puzzle ---
        puzzle_request = (
            "There are three boxes. One contains only apples, one contains only oranges, and one contains both. "
            "Each box is incorrectly labeled. You can only open one box and take out one fruit. "
            "By looking at the fruit, how can you label all the boxes correctly?"
        )
        puzzle_system_prompt = "You are a helpful assistant that excels at advanced logic puzzles and provides step-by-step reasoning."

        print(f"-> Agent Receives Task: {puzzle_request[:70]}...")

        # Run the agent task
        result = my_agent.run_task(puzzle_request, puzzle_system_prompt)

        # Log the full structured data for the observability pipeline
        log_agent_event(result["observability_data"])

        print("\n--- AGENT'S FINAL ANSWER ---")
        print(result["agent_output"])

        # Summarize key Agentic Metrics
        print("\n--- AGENTIC OBSERVABILITY METRICS SUMMARY ---")
        obs_data = result['observability_data']
        print(f"**Inference Latency:** {obs_data.get('inference_latency_ms', 'N/A')} ms")
        print(f"**Total Tokens Consumed (Cost):** {obs_data.get('tokens_total', 'N/A')}")
        print(f"**Used Reasoning/CoT:** {obs_data.get('had_reasoning_step', 'N/A')}")

        print("\nThis structured log is vital for monitoring **cost, performance, and decision quality** in a real AI Agent system.")

✅ DeepSeek Client Initialized successfully.
-> Agent Receives Task: There are three boxes. One contains only apples, one contains only ora...

✨ AGENT OBSERVABILITY TRACE CAPTURED (DeepSeek Agent) ✨
{
    "timestamp": "2025-12-04 11:25:39",
    "agent_id": "Reasoner-001",
    "request_summary": "There are three boxes. One contains only apples, o...",
    "inference_latency_ms": 85132.39,
    "tokens_input": 75,
    "tokens_output": 2441,
    "tokens_total": 2516,
    "model_name": "deepseek-reasoner",
    "had_reasoning_step": true,
    "response_quality_evaluation": "Requires Human or Automated Grading",
    "reasoning_trace_length": 8992
}

--- AGENT'S FINAL ANSWER ---
To solve the puzzle, open the box labeled "Both". Since all boxes are incorrectly labeled, this box cannot contain both fruits. It must contain either only apples or only oranges.

After taking out one fruit:
- If you take out an **apple**, then the box labeled "Both" actually contains **only apples**.  
  - Therefore,

## RUN2

In [2]:
import time
import json
from openai import OpenAI
from typing import Dict, Any
from google.colab import userdata # Keep this for Colab environment

# --- CONFIGURATION (Direct Key Retrieval) ---

try:
    API_KEY = userdata.get("DEEPSEEK_API_KEY")
    if not API_KEY:
        raise ValueError("DEEPSEEK_API_KEY not found in Colab userdata.")
    client = OpenAI(api_key=API_KEY, base_url="https://api.deepseek.com")
    print("✅ DeepSeek Client Initialized successfully for new run.")
except Exception as e:
    print(f"❌ Error initializing client: {e}")
    client = None

# --- 1. The Agent Class (Same as before) ---
class DeepSeekReasonerAgent:
    """A minimal agent that uses DeepSeek-Reasoner to solve a task and logs observability metrics."""

    def __init__(self, client: OpenAI, agent_id="Reasoner-002"): # Changed ID for new run
        self.client = client
        self.id = agent_id

    def run_task(self, request: str, system_prompt: str) -> Dict[str, Any]:
        """Executes the LLM call and captures observability metrics."""

        start_time = time.time()

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": request},
        ]

        agent_output = "API call failed."
        obs_data = {}

        try:
            response = self.client.chat.completions.create(
                model="deepseek-reasoner",
                messages=messages,
                stream=False
            )

            end_time = time.time()
            inference_latency_ms = (end_time - start_time) * 1000
            usage = response.usage
            final_content = response.choices[0].message.content
            reasoning = getattr(response.choices[0].message, 'reasoning_content', 'N/A')
            agent_output = final_content

            # --- (B) Structured Observability Data Payload ---
            obs_data = {
                "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
                "agent_id": self.id,
                "request_summary": request[:50] + "...",
                "inference_latency_ms": round(inference_latency_ms, 2),
                "tokens_input": usage.prompt_tokens,
                "tokens_output": usage.completion_tokens,
                "tokens_total": usage.total_tokens,
                "model_name": response.model,
                "had_reasoning_step": reasoning != 'N/A',
                "response_quality_evaluation": "Requires Human or Automated Grading",
                "reasoning_trace_length": len(reasoning) if reasoning != 'N/A' else 0,
            }

        except Exception as e:
            end_time = time.time()
            inference_latency_ms = (end_time - start_time) * 1000
            agent_output = f"API Error: {str(e)}"
            obs_data = {
                "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
                "agent_id": self.id,
                "request_summary": request[:50] + "...",
                "inference_latency_ms": round(inference_latency_ms, 2),
                "status": "FAILURE",
                "error_message": str(e),
                "tokens_total": 0,
            }

        return {
            "agent_output": agent_output,
            "observability_data": obs_data
        }

# --- 2. Observability Logging Function (Same as before) ---
def log_agent_event(data: Dict[str, Any]):
    """Prints the structured observability data in a readable JSON format."""
    print("\n" + "="*70)
    print("✨ AGENT OBSERVABILITY TRACE CAPTURED (DeepSeek Agent) - CONCISED RUN ✨")
    print("="*70)
    print(json.dumps(data, indent=4))
    print("="*70)

# --- 3. Demo Execution (New Prompt) ---
if __name__ == "__main__":
    if not client:
        print("\nDemo cannot run. Client initialization failed.")
    else:
        my_agent = DeepSeekReasonerAgent(client=client)

        puzzle_request = (
            "There are three boxes. One contains only apples, one contains only oranges, and one contains both. "
            "Each box is incorrectly labeled. You can only open one box and take out one fruit. "
            "By looking at the fruit, how can you label all the boxes correctly?"
        )

        # --- MODIFIED SYSTEM PROMPT for Concision ---
        new_system_prompt = (
            "You are a helpful assistant that solves logic puzzles. "
            "**Your response must be brief: provide only the final 3-step solution without verbose introductions or examples.**"
        )

        print(f"\n-> Agent Receives Task with **CONCISE** System Prompt.")

        # Run the agent task
        result = my_agent.run_task(puzzle_request, new_system_prompt)

        # Log the full structured data for the observability pipeline
        log_agent_event(result["observability_data"])

        print("\n--- AGENT'S CONCISE FINAL ANSWER ---")
        print(result["agent_output"])

        # Summarize key Agentic Metrics
        print("\n--- COMPARATIVE AGENTIC OBSERVABILITY METRICS SUMMARY ---")
        obs_data = result['observability_data']
        print(f"**New Latency:** {obs_data.get('inference_latency_ms', 'N/A')} ms")
        print(f"**New Total Tokens:** {obs_data.get('tokens_total', 'N/A')}")
        print("\n**Compare this to the previous run's 56738.03 ms latency and 1604 tokens.** The change in these metrics will show the impact of the prompt optimization.")

✅ DeepSeek Client Initialized successfully for new run.

-> Agent Receives Task with **CONCISE** System Prompt.

✨ AGENT OBSERVABILITY TRACE CAPTURED (DeepSeek Agent) - CONCISED RUN ✨
{
    "timestamp": "2025-12-04 11:22:09",
    "agent_id": "Reasoner-002",
    "request_summary": "There are three boxes. One contains only apples, o...",
    "inference_latency_ms": 45321.93,
    "tokens_input": 88,
    "tokens_output": 1274,
    "tokens_total": 1362,
    "model_name": "deepseek-reasoner",
    "had_reasoning_step": true,
    "response_quality_evaluation": "Requires Human or Automated Grading",
    "reasoning_trace_length": 5293
}

--- AGENT'S CONCISE FINAL ANSWER ---
1. Open the box labeled "apples and oranges" and remove one fruit.
2. If the fruit is an apple, that box actually contains only apples; if it's an orange, it contains only oranges.
3. From this, deduce: If the opened box had apples, then the box labeled "apples" contains oranges and the box labeled "oranges" contains both. If